In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
learned_data_path = '../processed_EEG_data/clean_learned_JoshIrby.csv'
not_learned_data_path = '../processed_EEG_data/clean_notLearned_JoshIrby.csv'

learned_data = pd.read_csv(learned_data_path, header = None)
not_learned_data = pd.read_csv(not_learned_data_path, header = None)

In [3]:
# checking how much data for each category
print(f"original learned shape: {learned_data.shape}")
print(f"original not_learned shape: {not_learned_data.shape}")

# make them equal to remove bias
learned_data = learned_data.sample(frac=1, random_state=42).reset_index(drop=True)
not_learned_data = not_learned_data.sample(frac=1, random_state=42).reset_index(drop=True)

# get length 
learned_length = learned_data.shape[0]
not_learned_length = not_learned_data.shape[0]

new_learned_data = learned_data.drop(index=range(not_learned_length, learned_length))
new_not_learned_data = not_learned_data.drop(index=range(learned_length, not_learned_length))

# print results
print(f"new learned shape: {new_learned_data.shape}")
print(f"new not_learned shape: {not_learned_data.shape}")

original learned shape: (141, 2456)
original not_learned shape: (21, 2456)
new learned shape: (21, 2456)
new not_learned shape: (21, 2456)


In [4]:
#Label Data

new_learned_data['label'] = 1
new_not_learned_data['label'] = 0

In [5]:
#Combine and Shuffle

full_data = pd.concat([new_learned_data, new_not_learned_data], ignore_index = True)
full_data = full_data.sample(frac = 1).reset_index(drop = True)

In [6]:
#Split between X and Y Data
#X is 42 rows (epochs) 21 Learned, 21 Not Learned
#Y is list with length 42 of 0 (NL) and 1 (L)

X = full_data.drop('label', axis = 1).values
Y = full_data['label'].values

In [7]:
#Standardize Features

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [8]:
#Split Data into Train and Test

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [9]:
#Building Neural Network Model

model = Sequential([
    Dense(8, activation = 'relu', input_shape = (X.shape[1],)),
    Dense(1, activation = 'sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.1
)

y_pred = model.predict(X_test)
y_pred_labels = (y_pred > 0.5).astype("int32")  # convert probabilities to 0 or 1

# Print accuracy
print("Accuracy:", accuracy_score(y_test, y_pred_labels))

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7119 - loss: 0.8558 - val_accuracy: 0.2500 - val_loss: 0.8691
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8434 - loss: 0.4843 - val_accuracy: 0.2500 - val_loss: 1.1267
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8685 - loss: 0.4758 - val_accuracy: 0.2500 - val_loss: 1.3236
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8894 - loss: 0.4316 - val_accuracy: 0.2500 - val_loss: 1.4255
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9332 - loss: 0.3332 - val_accuracy: 0.2500 - val_loss: 1.5137
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9562 - loss: 0.3042 - val_accuracy: 0.2500 - val_loss: 1.5422
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.2317 - val_accuracy: 0.2500 - val_loss: 1.5484
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9770 - loss: 0.1853 - val_accuracy: 0.2500 - val_loss: 1.5347
Epoch 9/20


In [11]:
clf = LogisticRegression()
scores = cross_val_score(clf, X, Y, cv=5)
print("Mean accuracy:", scores.mean())

Mean accuracy: 0.3111111111111111
